In [2]:
from utils_1.goenv import GoEnv,_format_state, create_env_copy
from utils_1.fnet import NeuralTrainer, AlphaNeural
import numpy as np
import goSim
from tqdm.auto import tqdm

In [3]:
class AlphaGoPlayer():
    def __init__(self, init_state, seed, player_color, path = "utils_1/nov8/net30.model",debug=True):
        # setup parameters
        self.init_state = init_state
        self.seed = seed
        self.player_color = player_color
        
        # init environment
        if player_color==1:
            self.env = GoEnv(player_color="black", board_size=13)
        else:
            self.env = GoEnv(player_color="white", board_size=13)
        self.env.reset()
        self.debug=debug
        
        # load neural network
        self.model = NeuralTrainer(res_blocks=10, board_size=13)
        self.model.load_model(path)
        
    def get_action(self, cur_state, opponent_action):
        if opponent_action>=0:
            self.env.step(opponent_action)
            if self.debug:
                print('-------------------------------------')
                print('opponent_action: ' + str(opponent_action))
                print('-------------------------------------')
        
        policy, value = self.model.predict(self.env.get_history())
        legal_moves = self.env.get_legal_moves()
        policy = policy*legal_moves
        action = np.argmax(policy)
        self.env.step(action)
        
        if self.debug:
            print('-------------------------------------')
            print('player_action: ' + str(action))
            print('-------------------------------------')
            self.env.print_board()
        
        return action

# Mankaran's implementation

In [7]:
vic = 0
matches = 100
for jk in tqdm(range(matches)):
    env_mank = GoEnv("black", board_size=13)
    s = env_mank.reset();
    opp_action=-1
    a1 = AlphaGoPlayer(s,1,1,"utils_nov8/nov8/net36.model", False)
    a2 = AlphaGoPlayer(s, 1, 2, "utils_nov8/nov8/net36.model", False)
#     a1 = AlphaGoPlayer(s,1,1,"utils_1/nov9/net11.model", False)
#     a2 = AlphaGoPlayer(s, 1, 2, "utils_nov8/nov8/net37.model", False)
    for i in range(400):
        if(i==399):
            print("Limit reached")
        s = env_mank.get_history()
        action = np.nonzero(env_mank.get_legal_moves())[0]
        action = np.random.choice(action)
        action = a1.get_action(s,opp_action)
        env_mank.step(action)
        if (env_mank.isComplete()):
#             print(action, opp_action)
#             print(env_mank.curr_score())
#             print(env_mank.get_winner())
#             print("Num_moves:" + str(i+1))
            break   

        opp_action = np.nonzero(env_mank.get_legal_moves())[0]
        opp_action = np.random.choice(opp_action)
#         opp_action = a2.get_action(s,action)
        env_mank.step(opp_action)
    #     print(action, opp_action)
    #     print(env_mank.curr_score())
    #     env_mank.print_board()
        if (env_mank.isComplete()):
#             print(env_mank.curr_score())
#             print(env_mank.get_winner())
#             print("Num_moves:" + str(i+1))
            break
#     print(env_mank.curr_score())
    if(env_mank.curr_score()<0):
        vic += 1
    if(jk%10==0):
        print(jk+1,vic/(jk+1))
#     print("Loop ended")
print(vic/matches)

1 0.0
11 0.7272727272727273
21 0.7619047619047619
31 0.7419354838709677
41 0.7804878048780488
51 0.803921568627451
61 0.819672131147541
71 0.8309859154929577
81 0.8024691358024691
91 0.8021978021978022

0.79


# My Implementation

In [9]:
players = {
    'black': 'a1',
    'white': 'a2'
}

wins = {
    'a1': 0,
    'a2': 0
}

matches = 100
    
for match_no in tqdm(range(matches)):
    state = GoEnv("black", board_size=13)
    s = state.reset()
    
    a1 = AlphaGoPlayer(s, 1, 1,"utils_nov8/nov8/net37.model", False)
    a2 = AlphaGoPlayer(s, 1, 2, "utils_1/nov9-correct/net11.model", False)

    a1_action = -1
    a2_action = -1
    
    for i in range(400):
        if(i==399):
            print("Limit reached")
            
#         stack = state.get_history()
        a1_action = a1.get_action('null', a2_action)
        state.step(a1_action)
        if (state.isComplete()):
            break   

        a2_action = a2.get_action('null', a1_action)
        state.step(a2_action)
        
        if (state.isComplete()):
            break
            
    winner = -1 * state.get_winner()
    if winner == 1:
        # It's black's victory
        wins[players['black']] += 1
    else:
        # It's white victory
        wins[players['white']] += 1
    
    if ((match_no + 1) % 10 == 0):
        # Log after every 10 matches
        print (wins)

print(wins)

{'a1': 10, 'a2': 0}
{'a1': 20, 'a2': 0}
{'a1': 30, 'a2': 0}
{'a1': 40, 'a2': 0}
{'a1': 50, 'a2': 0}
{'a1': 60, 'a2': 0}
{'a1': 70, 'a2': 0}
{'a1': 80, 'a2': 0}
{'a1': 90, 'a2': 0}
{'a1': 100, 'a2': 0}

{'a1': 100, 'a2': 0}
